In [1]:
import sys
from pathlib import Path
import json
import pandas as pd
import torch
from langchain.vectorstores import FAISS
from langchain_openai import OpenAIEmbeddings
from dotenv import load_dotenv
import os

# Append retrievers to the Python path (not just src)
sys.path.append(str(Path().resolve().parent / "src"))

# Import pipeline components
from retrievers.vectorrag.index_faiss import build_faiss_index_from_json
from retrievers.vectorrag.document_loader import load_json_documents
from retrievers.vectorrag.chunker import chunk_documents
from retrievers.vectorrag.embedder import init_embedder
from retrievers.vectorrag.index_faiss import build_faiss_index, load_faiss_index
from retrievers.vectorrag.retriever import rerank_search

from generator.generator import ChatGPTGenerator 
import numpy as np
import json
from retrievers.vectorrag.reranker import CrossEncoderReranker
import numpy as np


In [ ]:
import json

# Load enriched corpus
path = "/Users/alex/Documents/Data Science Master/thesis_RAG/data/Retriever_Context (Eval)/existing_embeddings_with_meta_data.jsonl"
docs = []

with open(path, 'r', encoding='utf-8') as f:
    for line in f:
        if line.strip():
            docs.append(json.loads(line))

# Quick check
print(f"Loaded {len(docs)} documents")
print(json.dumps(docs[0], indent=2))

Loaded 10130 documents
{
  "chunk_id": "row_0_chunk_0",
  "text": "Cboe Global Markets, Inc. and Subsidiaries\n\nConsolidated Statements of Income\n\nYears ended December 31, 2023, 2022, and 2021\n\n(In millions, except per share data)\n\n\n    \n\n2023\n\n    \n\n2022\n\n    \n\n2021\n\n \n\nRevenues:\n\nCash and spot markets\n\n\n$\n\n1,445.1\n\n\n$\n\n1,777.6\n\n\n$\n\n1,660.5\n\n\nData and access solutions\n\n539.2\n\n497.0\n\n427.7\n\n\nDerivatives markets\n\n\n \n\n1,789.2\n\n\n \n\n1,683.9\n\n\n \n\n1,406.6\n\n\nTotal revenues\n\n\n \n\n3,773.5\n\n\n \n\n3,958.5\n\n\n \n\n3,494.8\n\n\nCost of revenues:\n\n  Liquidity payments\n\n\n \n\n1,385.8\n\n\n \n\n1,670.2\n\n\n \n\n1,650.7\n\n\n  Routing and clearing\n\n79.1\n\n83.2\n\n87.8\n\n\n  Section 31 fees\n\n185.7\n\n329.8\n\n179.6\n\n\n  Royalty fees and other cost of revenues\n\n\n \n\n204.9\n\n\n \n\n133.6\n\n\n \n\n100.6\n\n\nTotal cost of revenues\n\n\n \n\n1,855.5\n\n\n \n\n2,216.8\n\n\n \n\n2,018.7\n\n\nRevenues less cost of

In [7]:
import json

# Paths to your files
file1 = '/Users/alex/Documents/Data Science Master/thesis_RAG/data/database_05_08/embedded_chunks.json'
file2 = '/Users/alex/Documents/Data Science Master/thesis_RAG/data/database_05_08/existing_embeddings_with_meta_data.jsonl'

# Inspect first few entries from embedded_chunks.json
print("📁 embedded_chunks.json")
with open(file1, 'r') as f:
    data_json = json.load(f)

print(f"Type: {type(data_json)}")
if isinstance(data_json, list):
    print(f"Loaded {len(data_json)} entries")
    print("Sample entry:")
    print(json.dumps(data_json[0], indent=2))
else:
    print("Top-level keys:", list(data_json.keys()))
    first_key = next(iter(data_json))
    print("Sample entry:")
    print(json.dumps(data_json[first_key], indent=2))

print("\n" + "-"*60 + "\n")

# Inspect first few entries from existing_embeddings_with_meta_data.jsonl
print("📁 existing_embeddings_with_meta_data.jsonl")
with open(file2, 'r') as f:
    for i, line in enumerate(f):
        entry = json.loads(line)
        print(f"Entry {i+1}:")
        print(json.dumps(entry, indent=2))
        if i >= 2:
            break  # just show 3 entries max

📁 embedded_chunks.json
Type: <class 'list'>
Loaded 10146 entries
Sample entry:
{
  "chunk_id": "row_0_chunk_0",
  "row_index": 0,
  "text": "Cboe Global Markets, Inc. and Subsidiaries\n\nConsolidated Statements of Income\n\nYears ended December 31, 2023, 2022, and 2021\n\n(In millions, except per share data)\n\n\n    \n\n2023\n\n    \n\n2022\n\n    \n\n2021\n\n \n\nRevenues:\n\nCash and spot markets\n\n\n$\n\n1,445.1\n\n\n$\n\n1,777.6\n\n\n$\n\n1,660.5\n\n\nData and access solutions\n\n539.2\n\n497.0\n\n427.7\n\n\nDerivatives markets\n\n\n \n\n1,789.2\n\n\n \n\n1,683.9\n\n\n \n\n1,406.6\n\n\nTotal revenues\n\n\n \n\n3,773.5\n\n\n \n\n3,958.5\n\n\n \n\n3,494.8\n\n\nCost of revenues:\n\n  Liquidity payments\n\n\n \n\n1,385.8\n\n\n \n\n1,670.2\n\n\n \n\n1,650.7\n\n\n  Routing and clearing\n\n79.1\n\n83.2\n\n87.8\n\n\n  Section 31 fees\n\n185.7\n\n329.8\n\n179.6\n\n\n  Royalty fees and other cost of revenues\n\n\n \n\n204.9\n\n\n \n\n133.6\n\n\n \n\n100.6\n\n\nTotal cost of revenues\n\n\n 

In [3]:
dimensions = set(len(item['embedding']) for item in embedded_data)

if len(dimensions) > 1:
    print("❌ Inconsistent embedding dimensions found:", dimensions)
else:
    print("✅ All embeddings have consistent dimension:", dimensions.pop())

✅ All embeddings have consistent dimension: 1536


In [8]:
import json

with open("/Users/alex/Documents/Data Science Master/thesis_RAG/data/database_05_08/embedded_chunks.json", "r") as f:
    embedded_chunks = json.load(f)

# Total number of chunks
num_chunks = len(embedded_chunks)

# Check embedding dimension of the first entry
first_vector = embedded_chunks[0]["embedding"]
embedding_dim = len(first_vector)

print(f"✅ Embedded Chunks: {num_chunks} chunks, each with dimension {embedding_dim}")

✅ Embedded Chunks: 10146 chunks, each with dimension 1536


In [9]:
dims = set(len(chunk["embedding"]) for chunk in embedded_chunks)
if len(dims) > 1:
    print("⚠️ Inconsistent embedding dimensions found:", dims)
else:
    print(f"✅ All chunks have consistent embedding dimension: {dims.pop()}")

✅ All chunks have consistent embedding dimension: 1536


In [11]:
import json

with open("/Users/alex/Documents/Data Science Master/thesis_RAG/data/database_05_08/merged_gold_eval_dataset.json", "r") as f:
    gold_data = json.load(f)

count_reference_contexts = 0
count_gold_context_str = 0
count_gold_context_dict = 0
total = len(gold_data)

for sample in gold_data:
    # Count reference_contexts
    ref = sample.get("reference_contexts", [])
    if isinstance(ref, list) and any(r.strip() for r in ref):
        count_reference_contexts += 1

    # Count gold_context if it's a non-empty string
    gold_ctx = sample.get("gold_context", "")
    if isinstance(gold_ctx, str) and gold_ctx.strip():
        count_gold_context_str += 1
    elif isinstance(gold_ctx, dict) and any(v.strip() for v in gold_ctx.values()):
        count_gold_context_dict += 1

print(f"🟩 Entries with non-empty reference_contexts: {count_reference_contexts}")
print(f"🟨 Entries with gold_context (string):        {count_gold_context_str}")
print(f"🟦 Entries with gold_context (dict):          {count_gold_context_dict}")
print(f"📊 Total entries:                            {total}")

🟩 Entries with non-empty reference_contexts: 50
🟨 Entries with gold_context (string):        0
🟦 Entries with gold_context (dict):          81
📊 Total entries:                            149


In [4]:
import json
import faiss
import numpy as np
import pickle

# Load embeddings from embedded_chunks.json
with open('/Users/alex/Documents/Data Science Master/thesis_RAG/data/database_05_08/embedded_chunks.json', 'r') as f:
    embedded_data = json.load(f)

# Create lists for vectors and IDs
vectors = []
ids = []
texts = []

for item in embedded_data:
    vectors.append(item['embedding'])
    ids.append(item['chunk_id'])
    texts.append(item['text'])

# Convert to float32 numpy array
embedding_matrix = np.array(vectors).astype('float32')

# 🔄 Normalize vectors to unit length (for cosine similarity)
faiss.normalize_L2(embedding_matrix)

# Use Inner Product index (cosine similarity after normalization)
dimension = embedding_matrix.shape[1]
index = faiss.IndexFlatIP(dimension)
index.add(embedding_matrix)

# Save FAISS index
faiss.write_index(index, 'faiss_index.idx')

# Load metadata from JSONL
metadata_dict = {}
with open('/Users/alex/Documents/Data Science Master/thesis_RAG/data/database_05_08/existing_embeddings_with_meta_data.jsonl', 'r') as f:
    for line in f:
        meta = json.loads(line)
        chunk_id = meta['chunk_id']
        metadata_dict[chunk_id] = meta

# Save chunk_id order and metadata
with open('retriever_metadata.pkl', 'wb') as f:
    pickle.dump({'chunk_ids': ids, 'metadata': metadata_dict}, f)

print("✅ FAISS cosine index and metadata saved successfully.")

✅ FAISS cosine index and metadata saved successfully.


In [2]:
import faiss
import numpy as np
import pandas as pd

# Initialize your embedder
embedder = init_embedder()

# Load FAISS index
index = faiss.read_index("/Users/alex/Documents/Data Science Master/thesis_RAG/data/database_05_08/faiss_index.idx")  # or your full path

# Load metadata
with open("/Users/alex/Documents/Data Science Master/thesis_RAG/data/database_05_08/retriever_metadata.pkl", "rb") as f:
    import pickle
    data = pickle.load(f)
    chunk_ids = data["chunk_ids"]
    metadata_dict = data["metadata"]

# Embed your query
query = "What was the operating income for Cboe Global Markets in 2023?"
query_embedding = embedder.embed_query(query)
query_embedding = np.array(query_embedding, dtype="float32").reshape(1, -1)
faiss.normalize_L2(query_embedding)  # Ensure cosine norm

# Search top-k
k = 5
D, I = index.search(query_embedding, k)

# Display results
print(f"\nTop {k} results for query: \"{query}\"\n")
for idx, score in zip(I[0], D[0]):
    chunk_id = chunk_ids[idx]
    metadata = metadata_dict[chunk_id]
    
    print(f"--- Rank {idx+1} ---")
    print(f"Score (cosine sim): {score:.4f}")
    print(f"Source: {metadata.get('source', 'N/A')}")
    print(f"Text:\n{metadata.get('text', '')[:700]}")
    print("-" * 80)


Top 5 results for query: "What was the operating income for Cboe Global Markets in 2023?"

--- Rank 1 ---
Score (cosine sim): 0.8826
Source: Finder
Text:
Cboe Global Markets, Inc. and Subsidiaries

Consolidated Statements of Income

Years ended December 31, 2023, 2022, and 2021

(In millions, except per share data)


    

2023

    

2022

    

2021

 

Revenues:

Cash and spot markets


$

1,445.1


$

1,777.6


$

1,660.5


Data and access solutions

539.2

497.0

427.7


Derivatives markets


 

1,789.2


 

1,683.9


 

1,406.6


Total revenues


 

3,773.5


 

3,958.5


 

3,494.8


Cost of revenues:

  Liquidity payments


 

1,385.8


 

1,670.2


 

1,650.7


  Routing and clearing

79.1

83.2

87.8


  Section 31 fees

185.7

329.8

179.6


  Royalty fees and other cost of revenues


 

204.9


 

133.6


 

100.6


Total cost of 
--------------------------------------------------------------------------------
--- Rank 881 ---
Score (cosine sim): 0.8424
Source: None
Text:


# Evaluation

In [5]:
from difflib import SequenceMatcher
import json

with open("/Users/alex/Documents/Data Science Master/thesis_RAG/data/database_05_08/merged_gold_eval_dataset.json", "r") as f:
    gold_data = json.load(f)


def fuzzy_match(a, b, threshold=0.85):
    return SequenceMatcher(None, a, b).ratio() >= threshold

def check_gold_context_coverage(reference_contexts, metadata_dict):
    for ref in reference_contexts:
        ref = ref.lower().strip()
        for chunk in metadata_dict.values():
            text = chunk.get("text", "").lower()
            if fuzzy_match(ref, text):
                return True
    return False

# Count how many reference contexts are covered
found_count = 0
total = 0

for sample in gold_data:
    gold_context = sample.get("gold_context", {})
    reference_contexts = []
    if isinstance(gold_context, dict):
        reference_contexts = list(gold_context.values())
    elif isinstance(gold_context, str) and gold_context.strip():
        reference_contexts = [gold_context.strip()]
    
    if reference_contexts:
        total += 1
        if check_gold_context_coverage(reference_contexts, metadata_dict):
            found_count += 1

print(f"\n📊 Gold context coverage in index: {found_count}/{total} ({found_count / total:.2%})")


📊 Gold context coverage in index: 0/81 (0.00%)


In [9]:
# Flatten all chunk texts for brute-force search
all_texts = [chunk["text"].lower() for chunk in metadata_dict.values()]

# Check if each gold context is substring of any chunk
def is_in_raw_chunks(context: str) -> bool:
    return any(context.lower() in text for text in all_texts)

count_found = 0
for sample in gold_data:
    gold_context = sample.get("gold_context", {})
    references = list(gold_context.values()) if isinstance(gold_context, dict) else [gold_context]
    for ref in references:
        if is_in_raw_chunks(ref):
            count_found += 1
            break  # only need one match to count the sample

print(f"✅ Raw chunk match found for {count_found}/{len(gold_data)} samples")

✅ Raw chunk match found for 118/201 samples


Problem: Gold context not covered in FAISS. 

In [6]:
import json
from difflib import SequenceMatcher

# Load gold contexts
with open("/Users/alex/Documents/Data Science Master/thesis_RAG/data/database_05_08/merged_gold_eval_dataset.json") as f:
    gold_data = json.load(f)

# Load existing embedded text chunks
with open("/Users/alex/Documents/Data Science Master/thesis_RAG/data/database_05_08/embedded_chunks.json") as f:
    embedded_chunks = json.load(f)

embedded_texts = [chunk["text"].lower().strip() for chunk in embedded_chunks]

# Fuzzy match gold context against embedded texts
def fuzzy_match(a, b, threshold=0.85):
    return SequenceMatcher(None, a.strip().lower(), b.strip().lower()).ratio() >= threshold

def context_is_embedded(reference_contexts):
    for ref in reference_contexts:
        for text in embedded_texts:
            if fuzzy_match(ref, text):
                return True
    return False

found = 0
total = 0

for sample in gold_data:
    gold_context = sample.get("gold_context", {})
    reference_contexts = []

    if isinstance(gold_context, dict):
        reference_contexts = list(gold_context.values())
    elif isinstance(gold_context, str) and gold_context.strip():
        reference_contexts = [gold_context.strip()]

    if reference_contexts:
        total += 1
        if context_is_embedded(reference_contexts):
            found += 1

print(f"\n📊 Gold context found in embedded chunks: {found}/{total} ({found/total:.2%})")


📊 Gold context found in embedded chunks: 0/81 (0.00%)


In [12]:
import json
from pathlib import Path
from difflib import SequenceMatcher

# === Load existing embedded chunks ===
jsonl_path = Path("/Users/alex/Documents/Data Science Master/thesis_RAG/data/database_05_08/existing_embeddings_with_meta_data.jsonl")
with open(jsonl_path) as f:
    existing_chunks = [json.loads(line) for line in f]

# Build normalized search index
all_texts = [chunk["text"].lower().strip() for chunk in existing_chunks]

def fuzzy_match(a, b, threshold=0.85):
    return SequenceMatcher(None, a, b).ratio() >= threshold

# === Load gold dataset ===
with open("/Users/alex/Documents/Data Science Master/thesis_RAG/data/database_05_08/merged_gold_eval_dataset.json") as f:
    gold_data = json.load(f)

# === Helper: Extract usable reference contexts ===
def get_reference_contexts(sample):
    ref = sample.get("reference_contexts", [])
    if isinstance(ref, list) and any(r.strip() for r in ref):
        return [r.strip().lower() for r in ref if isinstance(r, str) and r.strip()]
    
    gc = sample.get("gold_context", {})
    if isinstance(gc, dict):
        return [v.strip().lower() for v in gc.values() if isinstance(v, str) and v.strip()]
    
    return []

# === Check matches ===
found = 0
total = 0

for sample in gold_data:
    reference_contexts = get_reference_contexts(sample)

    if reference_contexts:
        total += 1
        for ref in reference_contexts:
            if any(fuzzy_match(ref, chunk) for chunk in all_texts):
                found += 1
                break  # Found at least one match for this sample

print(f"\n📦 Gold contexts matched in JSONL: {found}/{total} ({found / total:.2%})")


📦 Gold contexts matched in JSONL: 0/81 (0.00%)


In [13]:
for sample in gold_data[:3]:  # try more later
    print("\n🔍 Sample:", sample.get("ID", sample.get("question", "N/A")))
    for ctx in get_reference_contexts(sample):
        print("REF:", ctx[:100], "...")


🔍 Sample: 182f0809

🔍 Sample: c593f878

🔍 Sample: cb08b8b0


In [14]:
print("\n📦 First few embedded chunks:")
for chunk in all_texts[:3]:
    print("CHUNK:", chunk[:100], "...")


📦 First few embedded chunks:
CHUNK: cboe global markets, inc. and subsidiaries

consolidated statements of income

years ended december  ...
CHUNK: 15.6


  goodwill impairment

—

460.9

—


  other expenses

13.9

26.0

16.4


total operating exp ...
CHUNK: employees

as of december 31, 2023, we employed 1,647 individuals in the following locations:


loca ...


In [15]:
lengths = [len(chunk["text"]) for chunk in existing_chunks]
print("📏 Avg embedded chunk length:", sum(lengths) / len(lengths))

📏 Avg embedded chunk length: 1721.7633550167554


In [16]:
for s in gold_data:
    refs = get_reference_contexts(s)
    for r in refs:
        print("📄 Gold context length:", len(r))
        print(r)
        break
    break

In [5]:
from tqdm import tqdm
import numpy as np
import faiss
import pickle
import json
import sys
from pathlib import Path
import time
sys.path.append(str(Path().resolve().parent / "src"))
from retrievers.vectorrag.embedder import init_embedder

embedder = init_embedder()

# Load raw chunk metadata
with open("/Users/alex/Documents/Data Science Master/thesis_RAG/data/Retriever_Context (Eval)/existing_embeddings_with_meta_data.jsonl") as f:
    metadata_dict = {json.loads(line)["chunk_id"]: json.loads(line) for line in f}

chunk_ids = list(metadata_dict.keys())
all_texts = [metadata_dict[cid]["text"] for cid in chunk_ids]

# === Parameters ===
BATCH_SIZE = 100
dimension = 1536  # for ada-002

# === Init empty index
index = faiss.IndexFlatIP(dimension)
chunk_id_order = []

# === Stream embedding and indexing
for i in tqdm(range(0, len(all_texts), BATCH_SIZE), desc="Embedding + Indexing"):
    batch_ids = chunk_ids[i:i+BATCH_SIZE]
    batch_texts = all_texts[i:i+BATCH_SIZE]

    try:
        embeddings = embedder.embed_documents(batch_texts)
    except Exception as e:
        print(f"❌ Batch {i}-{i+BATCH_SIZE} failed: {e}")
        time.sleep(5)
        embeddings = embedder.embed_documents(batch_texts)

    emb_np = np.array(embeddings).astype("float32")
    faiss.normalize_L2(emb_np)

    index.add(emb_np)
    chunk_id_order.extend(batch_ids)

# ✅ Save final FAISS index and metadata
faiss.write_index(index, "faiss_index_full.idx")
with open("retriever_metadata_full.pkl", "wb") as f:
    pickle.dump({"chunk_ids": chunk_id_order, "metadata": metadata_dict}, f)

print("✅ FAISS + metadata written safely with streaming.")

Embedding + Indexing: 100%|██████████| 102/102 [02:45<00:00,  1.63s/it]

✅ FAISS + metadata written safely with streaming.


In [6]:
import pickle
import faiss

# Load FAISS index
index = faiss.read_index("/Users/alex/Documents/Data Science Master/thesis_RAG/data/Retriever_Context (Eval)/faiss_index_full.idx")

# Load metadata
with open("/Users/alex/Documents/Data Science Master/thesis_RAG/data/Retriever_Context (Eval)/retriever_metadata_full.pkl", "rb") as f:
    meta = pickle.load(f)

chunk_ids = meta["chunk_ids"]
metadata_dict = meta["metadata"]

# Sanity check
print(f"🔢 Index contains {index.ntotal} vectors")
print(f"🧠 Metadata contains {len(metadata_dict)} chunks")

🔢 Index contains 10130 vectors
🧠 Metadata contains 10130 chunks


In [11]:
import json

# Load gold data
with open("../data/data_processed/Train_Val_Test/gold_test_data_updated.json") as f:
    gold_data = json.load(f)

# Lowercased all chunk texts
all_texts = [chunk["text"].lower() for chunk in metadata_dict.values()]

# Match function
def is_in_indexed_chunks(context):
    return any(context.lower() in chunk_text for chunk_text in all_texts)

# Run test
count_found = 0
for sample in gold_data:
    gold_context = sample.get("gold_context", {})
    references = list(gold_context.values()) if isinstance(gold_context, dict) else [gold_context]
    for ref in references:
        if is_in_indexed_chunks(ref):
            count_found += 1
            break  # only one match needed per sample

print(f"✅ Gold context match found for {count_found}/{len(gold_data)} samples")

✅ Gold context match found for 118/201 samples


In [20]:
import asyncio
from typing import List, Dict, Any, Optional

from ragas import EvaluationDataset, evaluate
import os
from dotenv import load_dotenv

async def evaluate_ragas_dataset(
    dataset: List[Dict[str, Any]],
    metrics_list: Optional[List[str]] = None,
    llm_model: str = "gpt-4o-2024-11-20",
    llm_type: str = "openai",  # 'openai' or 'vllm'
    vllm_base_url: str = "http://localhost:8000/v1"
):
    """
    Evaluate a dataset using RAGAS with the new API.

    Args:
        dataset: List of dicts with keys: user_input, retrieved_contexts, response, reference
        metrics_list: List of metric names to compute (see available_metrics below). If None, all are used.
        llm_model: Model name (e.g., 'gpt-4o', 'gpt-3.5-turbo', 'Fin-R1')
        llm_type: 'openai' (default) or 'vllm'
        vllm_base_url: Base URL for vllm server (if using vllm)
    Returns:
        Dictionary of metric results
    """
    # Metric mapping
    from ragas.metrics import (
        LLMContextPrecisionWithReference,
        NonLLMContextPrecisionWithReference,
        LLMContextRecall,
        NonLLMContextRecall,
        ContextEntityRecall,
        Faithfulness,
        AnswerAccuracy,
        StringPresence,
    )
    available_metrics = {
        "context_precision_llm": LLMContextPrecisionWithReference,
        "context_precision_nonllm": NonLLMContextPrecisionWithReference,
        "context_recall_llm": LLMContextRecall,
        "context_recall_nonllm": NonLLMContextRecall,
        "context_entity_recall": ContextEntityRecall,
        "faithfulness": Faithfulness,
        "answer_accuracy": AnswerAccuracy,
        "string_presence": StringPresence,
    }
    # If no metrics_list, use all
    if metrics_list is None:
        metrics_list = list(available_metrics.keys())
    # Instantiate metrics
    metrics = [available_metrics[name]() for name in metrics_list if name in available_metrics]
    if not metrics:
        raise ValueError("No valid metrics selected.")

    load_dotenv()
    if llm_type == "openai":
        from langchain_openai import ChatOpenAI
        from ragas.llms import LangchainLLMWrapper
        OPENAI_API_KEY = os.getenv("OPENAI_API_KEY")
        if not OPENAI_API_KEY:
            raise ValueError("OPENAI_API_KEY not found in .env file.")
        llm = ChatOpenAI(
            model=llm_model,
            api_key=OPENAI_API_KEY,
            max_tokens=1024,         # or more, depending on context length
            timeout=60               # increase timeout to 60s
        )
        evaluator_llm = LangchainLLMWrapper(llm)
    elif llm_type == "vllm":
        from langchain_community.llms import VLLMOpenAI
        from ragas.llms import LangchainLLMWrapper
        llm = VLLMOpenAI(model=llm_model, base_url=vllm_base_url)
        evaluator_llm = LangchainLLMWrapper(llm)
    else:
        raise ValueError(f"Unknown llm_type: {llm_type}")

    evaluation_dataset = EvaluationDataset.from_list(dataset)
    return evaluate(
        dataset=evaluation_dataset,
        metrics=metrics,
        llm=evaluator_llm
    )


In [ ]:
finqa_eval_detailed

In [12]:
import json
from pathlib import Path

# === Load gold test data ===
GOLD_PATH = Path("../data/data_processed/Train_Val_Test/gold_test_data_updated.json")
with open(GOLD_PATH) as f:
    gold_data = json.load(f)

# === Lowercase all chunk texts for matching ===
all_texts = [chunk["text"].lower() for chunk in metadata_dict.values()]

# === Helper: match function ===
def is_in_indexed_chunks(context):
    return any(context.lower() in chunk_text for chunk_text in all_texts)

# === Filter matching samples ===
filtered_eval_dataset = []
count_found = 0

for sample in gold_data:
    gold_context = sample.get("gold_context", {})
    references = list(gold_context.values()) if isinstance(gold_context, dict) else [gold_context]

    for ref in references:
        if is_in_indexed_chunks(ref):
            sample["reference_contexts"] = references
            filtered_eval_dataset.append(sample)
            count_found += 1
            break  # only one match needed per sample

# === Report + Save ===
print(f"✅ Gold context match found for {count_found}/{len(gold_data)} samples")

with open("filtered_gold_eval_dataset.json", "w") as f:
    json.dump(filtered_eval_dataset, f, indent=2)

print("✅ Saved filtered dataset to filtered_gold_eval_dataset.json")

✅ Gold context match found for 118/201 samples
✅ Saved filtered dataset to filtered_gold_eval_dataset.json


AttributeError: 'list' object has no attribute 'head'

In [17]:
import json
import numpy as np
import faiss
import pickle
from pathlib import Path
import pandas as pd
from tqdm import tqdm


# === Setup Paths ===
BASE_DIR = Path("/Users/alex/Documents/Data Science Master/thesis_RAG/data/database_05_08")
FAISS_PATH = BASE_DIR / "faiss_index.idx"
META_PATH = BASE_DIR / "retriever_metadata.pkl"
GOLD_PATH = Path("/Users/alex/Documents/Data Science Master/thesis_RAG/data/database_05_08/merged_gold_eval_dataset.json")

# === Load FAISS Index and Metadata ===
with open(META_PATH, "rb") as f:
    data = pickle.load(f)
    chunk_ids = data["chunk_ids"]
    metadata_dict = data["metadata"]

index = faiss.read_index(str(FAISS_PATH))

# === Load your OpenAI embedder ===
embedder = init_embedder()

# === Load Gold QA Dataset ===
with open(GOLD_PATH) as f:
    gold_data = json.load(f)

# === Evaluation Loop ===
def evaluate_retriever(gold_data, embedder, top_k=50):
    results = []

    for sample in tqdm(gold_data):
        question = sample["question"]
        gold_context_raw = sample.get("gold_context", {})

        # Normalize gold context
        gold_contexts = list(gold_context_raw.values()) if isinstance(gold_context_raw, dict) else [gold_context_raw]
        gold_contexts = [g.lower().strip() for g in gold_contexts if isinstance(g, str)]

        # Embed and search
        try:
            q_embed = np.array(embedder.embed_query(question)).astype("float32").reshape(1, -1)
            faiss.normalize_L2(q_embed)
            D, I = index.search(q_embed, top_k)
        except Exception as e:
            print(f"Embedding failed for question: {question}\nError: {e}")
            continue

        retrieved_chunks = [metadata_dict[chunk_ids[i]]["text"].lower() for i in I[0]]
        found = any(g in r for g in gold_contexts for r in retrieved_chunks)

        results.append({
            "question": question,
            "gold_found_in_top_k": found,
            "gold_context": gold_contexts[0][:120] if gold_contexts else None,
            "first_hit_index": next((i for i, r in enumerate(retrieved_chunks) if any(g in r for g in gold_contexts)), -1)
        })

    return pd.DataFrame(results)

# === Run Evaluation ===
df = evaluate_retriever(gold_data, embedder=embedder, top_k=50)
df.to_csv("retriever_eval.csv", index=False)

# === Print Summary ===
total = len(df)
hits = (df["gold_found_in_top_k"] == True).sum()
print(f"\nRetriever Recall@50: {hits}/{total} → {100 * hits / total:.2f}%")

100%|██████████| 149/149 [01:03<00:00,  2.34it/s]


Retriever Recall@50: 114/149 → 76.51%


In [2]:
import json
import faiss
import numpy as np
import pickle
import pandas as pd
from pathlib import Path
import asyncio


# === Paths ===
BASE_DIR = Path("/Users/alex/Documents/Data Science Master/thesis_RAG/data/database_05_08")
FAISS_PATH = BASE_DIR / "faiss_index.idx"
META_PATH = BASE_DIR / "retriever_metadata.pkl"
GOLD_PATH = Path("/Users/alex/Documents/Data Science Master/thesis_RAG/data/database_05_08/merged_gold_eval_dataset.json")

# === Load Metadata and FAISS Index ===
with open(META_PATH, "rb") as f:
    data = pickle.load(f)
    chunk_ids = data["chunk_ids"]
    metadata_dict = data["metadata"]

index = faiss.read_index(str(FAISS_PATH))

# === Load Embedder and Generator ===
embedder = init_embedder()
generator = ChatGPTGenerator()
reranker = CrossEncoderReranker()

# === Load Gold Test Data ===
with open(GOLD_PATH) as f:
    gold_data = json.load(f)

# === Generate Evaluation Dataset ===
finqa_eval_dataset = []
finder_eval_dataset = []
finqa_count = 0
finder_count = 0

for sample in gold_data:
    if finqa_count >= 2 and finder_count >= 2:
        break

    question = sample["question"]
    reference = sample["answer"]
    gold_context = sample.get("gold_context", {})

    # Embed and retrieve
    query_embedding = np.array(embedder.embed_query(question), dtype="float32").reshape(1, -1)
    faiss.normalize_L2(query_embedding)
    D, I = index.search(query_embedding, 50)

    candidate_docs = [
        metadata_dict[chunk_ids[i]]
        for i in I[0]
        if chunk_ids[i] in metadata_dict and "text" in metadata_dict[chunk_ids[i]]
    ]
    reranked_docs = reranker.rerank(question, candidate_docs, top_k=10)

    # Generate answer
    response = generator.generate(question, [doc["text"] for doc in reranked_docs])

    # Prepare record for RAGAS
    record = {
        "user_input": question,
        "retrieved_contexts": [doc["text"] for doc in reranked_docs],
        "response": response,
        "reference": reference,
    }

    # === Check if reference context exists in indexed corpus ===
    reference_contexts = []
    if isinstance(gold_context, dict):
        reference_contexts = list(gold_context.values())
    elif isinstance(gold_context, str) and gold_context.strip():
        reference_contexts = [gold_context.strip()]

    if reference_contexts:
        record["reference_contexts"] = reference_contexts
        found = False
        for ref in reference_contexts:
            ref_clean = ref.lower().strip()
            for chunk_data in metadata_dict.values():
                if ref_clean in chunk_data.get("text", "").lower():
                    found = True
                    break
            if found:
                break
        if found:
            print("✅ Reference context FOUND in indexed chunks.")
        else:
            print("❌ Reference context NOT found in indexed chunks.")

    # === Append to dataset ===
    if reference_contexts and finqa_count < 2:
        finqa_eval_dataset.append(record)
        finqa_count += 1
    elif not reference_contexts and finder_count < 2:
        finder_eval_dataset.append(record)
        finder_count += 1

✅ Reference context FOUND in indexed chunks.
✅ Reference context FOUND in indexed chunks.


In [19]:
import re
def normalize_text(text):
    text = text.lower()
    text = re.sub(r'[^a-z0-9\s\.\-]', ' ', text)  # remove special chars
    text = re.sub(r'\s+', ' ', text)  # collapse whitespace
    return text.strip()

In [ ]:
from typing import List, Dict
import pandas as pd
import numpy as np
from tqdm import tqdm
from sklearn.metrics.pairwise import cosine_similarity
import spacy
from typing import List, Dict
import pandas as pd
from tqdm import tqdm
from openai import OpenAI
from typing import Tuple
import numpy as np
import pandas as pd
from typing import List, Dict
from sklearn.metrics.pairwise import cosine_similarity
from tqdm import tqdm
import re


import re
def normalize_text(text):
    text = text.lower()
    text = re.sub(r'[^a-z0-9\s\.\-]', ' ', text)  # remove special chars
    text = re.sub(r'\s+', ' ', text)  # collapse whitespace
    return text.strip()

def context_recall_with_reference(
    dataset: List[Dict],
    embedder,
    top_k_context_key: str = "retrieved_contexts",
    gold_context_key: str = "reference_contexts",
    method: str = "embedding",  # "exact", "embedding", or "both"
    sim_threshold: float = 0.85
) -> pd.DataFrame:
    """
    Evaluates context recall via exact match or embedding-based similarity.
 
    Args:
        dataset: List of QA samples with user_input, retrieved_contexts, reference_contexts
        embedder: an object with .embed_documents() method (e.g., OpenAIEmbedder or SentenceTransformer)
        method: "exact", "embedding", or "both"
        sim_threshold: similarity threshold for embedding-based match
 
    Returns:
        pd.DataFrame with recall scores and detailed diagnostics
    """
    results = []
 
    for sample in tqdm(dataset):
        question = sample.get("user_input", "N/A")
        retrieved = sample.get(top_k_context_key, [])
        gold = sample.get(gold_context_key, [])
 
        retrieved = [normalize_text(r) for r in retrieved if isinstance(r, str)]
        gold = [normalize_text(g) for g in gold if isinstance(g, str)]
 
        em_hits = 0
        emb_hits = 0
 
        # === Exact Match ===
        if method in {"exact", "both"}:
            from rapidfuzz import fuzz
            em_hits = sum(
                any(fuzz.partial_ratio(g, r) > 85 for r in retrieved) for g in gold
            )
 
        # === Embedding Similarity ===
        if method in {"embedding", "both"} and gold and retrieved:
            try:
                gold_emb = np.array(embedder.embed_documents(gold)).astype("float32")
                retrieved_emb = np.array(embedder.embed_documents(retrieved)).astype("float32")
                sim_matrix = cosine_similarity(gold_emb, retrieved_emb)
                emb_hits = sum(np.any(sim_row >= sim_threshold) for sim_row in sim_matrix)
            except Exception as e:
                print(f"Embedding error for question: {question}\n{e}")
 
        # Pick which metric to use for recall
        total = len(gold)
        recall_em = em_hits / total if total else 0
        recall_emb = emb_hits / total if total else 0
 
        results.append({
            "question": question,
            "total_gold": total,
            "recall_em": recall_em,
            "recall_emb": recall_emb,
            "missing_em": [g for g in gold if not any(g in r for r in retrieved)],
            "missing_emb": [g for idx, g in enumerate(gold)
                            if method in {"embedding", "both"} and
                            (idx >= len(sim_matrix) or not np.any(sim_matrix[idx] >= sim_threshold))]
        })
 
    return pd.DataFrame(results)

In [5]:

embedder = init_embedder()

df_results = context_recall_with_reference(
    dataset=finqa_eval_dataset,               # test only 10 samples
    embedder=embedder,    # OpenAIEmbedder(), SentenceTransformer, etc.
    method="both",                      # or "exact", "embedding"
    sim_threshold=0.85                  # optional
)

import pandas as pd
pd.set_option("display.max_colwidth", None)
print(df_results[["question", "recall_em", "recall_emb"]])

100%|██████████| 2/2 [00:01<00:00,  1.16it/s]

                                                                                                                                                                                                                 question  \
0  Q: For GPN, what was the fair value of share awards vested in 2009?\nA: 6.2\nQ: what was the value in 2007?\nA: 1.7\nQ: what was the net change in value?\nA: A0\nQ: what is the net change divided by the 2007 value?   
1                                                                                                                                             what was the cost per tower in American Tower’s colombia movil acquisition?   

   recall_em  recall_emb  
0        1.0         1.0  
1        1.0         1.0  


In [8]:
finqa_eval_dataset_df = pd.DataFrame(finqa_eval_dataset)

In [9]:
finqa_eval_dataset_df

user_input  \
0  Q: For GPN, what was the fair value of share awards vested in 2009?\nA: 6.2\nQ: what was the value in 2007?\nA: 1.7\nQ: what was the net change in value?\nA: A0\nQ: what is the net change divided by the 2007 value?   
1                                                                                                                                             what was the cost per tower in American Tower’s colombia movil acquisition?   

                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                       

# test with fuzzy matching and normalization

In [26]:
import json
import numpy as np
import faiss
import pickle
from pathlib import Path

# === Setup paths ===
BASE_DIR = Path("/Users/alex/Documents/Data Science Master/thesis_RAG/data/database_05_08")
FAISS_PATH = BASE_DIR / "faiss_index.idx"
META_PATH = BASE_DIR / "retriever_metadata.pkl"
GOLD_PATH = Path("/Users/alex/Documents/Data Science Master/thesis_RAG/data/database_05_08/merged_gold_eval_dataset.json")

# === Load index and metadata ===
with open(META_PATH, "rb") as f:
    meta = pickle.load(f)
    chunk_ids = meta["chunk_ids"]
    metadata_dict = meta["metadata"]

index = faiss.read_index(str(FAISS_PATH))

# === Load embedder, reranker, generator ===
embedder = init_embedder()
reranker = CrossEncoderReranker()
generator = ChatGPTGenerator()

# === Load sample 6 (index 5) ===
with open(GOLD_PATH) as f:
    gold_data = json.load(f)
sample = gold_data[5]

# === Extract question and reference ===
question = sample["question"]
reference = sample["answer"]
gold_context = sample.get("gold_context", {})

# === Step 1: Embed query and retrieve ===
query_emb = np.array(embedder.embed_query(question), dtype="float32").reshape(1, -1)
faiss.normalize_L2(query_emb)
D, I = index.search(query_emb, 50)

retrieved_chunks = [
    metadata_dict[chunk_ids[i]]
    for i in I[0]
    if chunk_ids[i] in metadata_dict and "text" in metadata_dict[chunk_ids[i]]
]

# === Step 2: Rerank retrieved chunks ===
reranked_chunks = reranker.rerank(question, retrieved_chunks, top_k=10)
retrieved_texts = [doc["text"] for doc in reranked_chunks]

# === Step 3: Generate response ===
response = generator.generate(question, retrieved_texts)

# === Step 4: Construct reference contexts ===
reference_contexts = []
if isinstance(gold_context, dict):
    reference_contexts = list(gold_context.values())
elif isinstance(gold_context, str) and gold_context.strip():
    reference_contexts = [gold_context.strip()]

# === Step 5: Assemble final RAGAS record ===
record = {
    "user_input": question,
    "retrieved_contexts": retrieved_texts,
    "response": response,
    "reference": reference,
    "reference_contexts": reference_contexts
}

# Optional: Print confirmation
print("✅ Created record with", len(reference_contexts), "reference contexts.")

✅ Created record with 2 reference contexts.


In [27]:
record

{'user_input': 'Q: what is the implicit interest cost rate reported by Entergy in 2016?\nA: 5.13\nQ: what is the rate divided by 100?\nA: A0\nQ: what is the value of lease payments due after 2021?\nA: 257812\nQ: what is the product of the rate and those payments?',
 'retrieved_contexts': ["['entergy corporation and subsidiaries notes to financial statements ferc audit report , system energy initially recorded as a net regulatory asset the difference between the recovery of the lease payments and the amounts expensed for interest and depreciation and continues to record this difference as a regulatory asset or liability on an ongoing basis , resulting in a zero net balance for the regulatory asset at the end of the lease term .', 'the amount was a net regulatory asset ( liability ) of ( $ 2.0 ) million and $ 60.6 million as of december 31 , 2011 and 2010 , respectively .', 'as of december 31 , 2011 , system energy had future minimum lease payments ( reflecting an implicit rate of 5.13% 

In [33]:
df_results = context_recall_with_reference(
    dataset=[record],
    embedder=embedder,                  # your embedder with .embed_documents()
    top_k_context_key="retrieved_contexts",
    gold_context_key="reference_contexts",
    method="both",                      # use "embedding", "exact", or "both"
    sim_threshold=0.75)

100%|██████████| 1/1 [00:01<00:00,  1.09s/it]


In [34]:
df_results 

,question,total_gold,recall_em,recall_emb,missing_em,missing_emb
0,Q: what is the implicit interest cost rate reported by Entergy in 2016?\nA: 5.13\nQ: what is the rate divided by 100?\nA: A0\nQ: what is the value of lease payments due after 2021?\nA: 257812\nQ: what is the product of the rate and those payments?,2,0.5,1.0,[the years thereafter of amount in thousands is 257812],[]


In [10]:
def compute_ndcg(
    dataset: List[Dict],
    embedder = None,
    top_k_context_key: str = "retrieved_contexts",
    gold_context_key: str = "reference_contexts",
    method: str = "embedding",  # "exact" or "embedding"
    sim_threshold: float = 0.85,
    k: int = 10
) -> pd.DataFrame:
    """
    Compute nDCG@k for retrieved contexts.
 
    Returns:
        DataFrame with nDCG@k per sample and average
    """
    results = []
 
    for sample in tqdm(dataset):
        question = sample.get("user_input", "N/A")
        retrieved = sample.get(top_k_context_key, [])[:k]
        gold = sample.get(gold_context_key, [])
 
        retrieved = [r.lower().strip() for r in retrieved if isinstance(r, str)]
        gold = [g.lower().strip() for g in gold if isinstance(g, str)]
 
        relevance = [0] * len(retrieved)  # Relevance at each rank
 
        if method == "exact":
            for i, r in enumerate(retrieved):
                if any(g in r for g in gold):
                    relevance[i] = 1
 
        elif method == "embedding" and embedder and gold and retrieved:
            try:
                gold_emb = np.array(embedder.embed_documents(gold)).astype("float32")
                retrieved_emb = np.array(embedder.embed_documents(retrieved)).astype("float32")
                sim_matrix = cosine_similarity(retrieved_emb, gold_emb)
                for i, sim_row in enumerate(sim_matrix):
                    if np.any(sim_row >= sim_threshold):
                        relevance[i] = 1
            except Exception as e:
                print(f"Embedding error for question: {question}\n{e}")
 
        # === Compute DCG and nDCG ===
        dcg = sum(rel / np.log2(i + 2) for i, rel in enumerate(relevance))  # +2 because log2(i+1) with 0-indexing
        ideal_relevance = sorted(relevance, reverse=True)
        idcg = sum(rel / np.log2(i + 2) for i, rel in enumerate(ideal_relevance))
 
        ndcg = dcg / idcg if idcg > 0 else 0.0
 
        results.append({
            "question": question,
            "dcg": dcg,
            "idcg": idcg,
            "ndcg@{}".format(k): ndcg
        })
 
    df = pd.DataFrame(results)
    df["mean_ndcg"] = df["ndcg@{}".format(k)].mean()
    return df

In [11]:
# Run nDCG evaluation on FinQA
ndcg_df = compute_ndcg(
    dataset=finqa_eval_dataset,
    embedder=embedder,             # must be the same one you used for context recall
    method="embedding",            # or "exact"
    sim_threshold=0.85,
    k=10                            # evaluates nDCG@10
)

# Display per-sample nDCG and the mean
print(ndcg_df[["question", "ndcg@10"]])
print(f"\n📊 Mean nDCG@10: {ndcg_df['ndcg@10'].mean():.4f}")

100%|██████████| 2/2 [00:01<00:00,  1.19it/s]

                                                                                                                                                                                                                 question  \
0  Q: For GPN, what was the fair value of share awards vested in 2009?\nA: 6.2\nQ: what was the value in 2007?\nA: 1.7\nQ: what was the net change in value?\nA: A0\nQ: what is the net change divided by the 2007 value?   
1                                                                                                                                             what was the cost per tower in American Tower’s colombia movil acquisition?   

    ndcg@10  
0  0.967468  
1  0.630930  

📊 Mean nDCG@10: 0.7992


### Create Evaluation Dataset

In [48]:
import json
import numpy as np
import faiss
import pickle
from pathlib import Path
from tqdm import tqdm

# === Setup paths ===
BASE_DIR = Path("/Users/alex/Documents/Data Science Master/thesis_RAG/data/database_05_08")
FAISS_PATH = BASE_DIR / "faiss_index.idx"
META_PATH = BASE_DIR / "retriever_metadata.pkl"
GOLD_PATH = BASE_DIR / "merged_gold_eval_dataset.json"
EMBED_SAVE_PATH = BASE_DIR / "query_embeddings.npy"
RAGAS_SAVE_PATH = BASE_DIR / "ragas_eval_dataset.json"

# === Load index and metadata ===
with open(META_PATH, "rb") as f:
    meta = pickle.load(f)
    chunk_ids = meta["chunk_ids"]
    metadata_dict = meta["metadata"]

index = faiss.read_index(str(FAISS_PATH))

# === Load embedder, reranker, generator ===
embedder = init_embedder()
reranker = CrossEncoderReranker()
generator = ChatGPTGenerator()

# === Load full dataset ===
with open(GOLD_PATH) as f:
    gold_data = json.load(f)

# === Track outputs ===
ragas_dataset = []
query_embeddings = []

# === Loop over all samples ===
for idx, sample in tqdm(enumerate(gold_data), total=len(gold_data)):
    question = sample["question"]
    reference = sample["answer"]
    gold_context = sample.get("gold_context", {})

    # === Step 1: Embed and save query ===
    query_emb = np.array(embedder.embed_query(question), dtype="float32").reshape(1, -1)
    faiss.normalize_L2(query_emb)
    query_embeddings.append(query_emb[0])  # flatten for easier saving

    # === Step 2: FAISS search ===
    D, I = index.search(query_emb, 50)
    retrieved_chunks = [
        metadata_dict[chunk_ids[i]]
        for i in I[0]
        if chunk_ids[i] in metadata_dict and "text" in metadata_dict[chunk_ids[i]]
    ]

    # === Step 3: Rerank ===
    reranked_chunks = reranker.rerank(question, retrieved_chunks, top_k=10)
    retrieved_texts = [doc["text"] for doc in reranked_chunks]

    # === Step 4: Generate answer ===
    # response = generator.generate(question, retrieved_texts)

    # === Step 5: Gold reference cleanup ===
    reference_contexts = []
    if isinstance(gold_context, dict):
        reference_contexts = list(gold_context.values())
    elif isinstance(gold_context, str) and gold_context.strip():
        reference_contexts = [gold_context.strip()]

    # === Step 6: Create record ===
    record = {
        "id": idx,
        "source": sample.get("source", "Unknown"),
        "user_input": question,
        "retrieved_contexts": retrieved_texts,
        # "response": response,
        "reference": reference,
        "reference_contexts": reference_contexts
    }
    ragas_dataset.append(record)

# === Save final RAGAS records ===
with open(RAGAS_SAVE_PATH, "w") as f:
    json.dump(ragas_dataset, f, indent=2)

# === Save query embeddings ===
np.save(EMBED_SAVE_PATH, np.array(query_embeddings))

  1%|▏         | 2/149 [00:14<17:40,  7.21s/it]


KeyboardInterrupt: 

In [37]:
import json
import numpy as np
import faiss
import pickle
from pathlib import Path
from tqdm import tqdm

# === Setup paths ===
BASE_DIR = Path("/Users/alex/Documents/Data Science Master/thesis_RAG/data/database_05_08")
FAISS_PATH = BASE_DIR / "faiss_index.idx"
META_PATH = BASE_DIR / "retriever_metadata.pkl"
GOLD_PATH = BASE_DIR / "merged_gold_eval_dataset.json"
EMBED_SAVE_PATH = BASE_DIR / "query_embeddings.npy"
RAGAS_SAVE_PATH = BASE_DIR / "ragas_eval_dataset_with_embeddings.json"

# === Load index and metadata ===
with open(META_PATH, "rb") as f:
    meta = pickle.load(f)
    chunk_ids = meta["chunk_ids"]
    metadata_dict = meta["metadata"]

index = faiss.read_index(str(FAISS_PATH))

# === Load embedder, reranker, generator ===
embedder = init_embedder()
reranker = CrossEncoderReranker()
generator = ChatGPTGenerator()

# === Load full dataset ===
with open(GOLD_PATH) as f:
    gold_data = json.load(f)

# === Track outputs ===
ragas_dataset = []
query_embeddings = []

# === Loop over all samples ===
for idx, sample in tqdm(enumerate(gold_data), total=len(gold_data)):
    question = sample["question"]
    reference = sample["answer"]
    gold_context = sample.get("gold_context", {})

    # === Step 1: Embed and save query ===
    query_emb = np.array(embedder.embed_query(question), dtype="float32").reshape(1, -1)
    faiss.normalize_L2(query_emb)
    query_embeddings.append(query_emb[0])  # flattened
     
    # === Step 2: FAISS search ===
    D, I = index.search(query_emb, 50)
    retrieved_chunks = [
        metadata_dict[chunk_ids[i]]
        for i in I[0]
        if chunk_ids[i] in metadata_dict and "text" in metadata_dict[chunk_ids[i]]
    ]

    # === Step 3: Rerank ===
    reranked_chunks = reranker.rerank(question, retrieved_chunks, top_k=10)
    retrieved_texts = [doc["text"] for doc in reranked_chunks]

    # === Step 4: Embed retrieved contexts ===
    retrieved_embeddings = embedder.embed_documents(retrieved_texts)

    # === Step 5: Prepare reference contexts and embed ===
    reference_contexts = []
    if isinstance(gold_context, dict):
        reference_contexts = list(gold_context.values())
    elif isinstance(gold_context, str) and gold_context.strip():
        reference_contexts = [gold_context.strip()]
    else:
        reference_contexts = []

    reference_embeddings = (
        embedder.embed_documents(reference_contexts) if reference_contexts else []
    )

    # === Step 6: Create final RAGAS record ===
    record = {
        "user_input": question,
        "retrieved_contexts": retrieved_texts,
        "retrieved_embeddings": retrieved_embeddings,
        "reference_contexts": reference_contexts,
        "reference_embeddings": reference_embeddings,
        "reference": reference
        # Optional: add response later if needed
    }

    ragas_dataset.append(record)

# === Save RAGAS records to JSON ===
with open(RAGAS_SAVE_PATH, "w") as f:
    json.dump(ragas_dataset, f, indent=2)

# === Save query embeddings separately as .npy ===
np.save(EMBED_SAVE_PATH, np.array(query_embeddings))

 11%|█▏        | 17/149 [00:54<07:06,  3.23s/it]


KeyboardInterrupt: 

In [ ]:
from typing import List, Dict, Tuple
import pandas as pd
import numpy as np
from tqdm import tqdm
from sklearn.metrics.pairwise import cosine_similarity
import re
from rapidfuzz import fuzz

def normalize_text(text):
    text = text.lower()
    text = re.sub(r'[^a-z0-9\s\.\-]', ' ', text)
    text = re.sub(r'\s+', ' ', text)
    return text.strip()

def context_recall_with_reference(
    dataset: List[Dict],
    embedder,
    top_k_context_key: str = "retrieved_contexts",
    gold_context_key: str = "reference_contexts",
    method: str = "embedding",  # "exact", "embedding", or "both"
    sim_threshold: float = 0.85
) -> Tuple[pd.DataFrame, pd.DataFrame]:
    """
    Evaluates context recall via exact match or embedding-based similarity.
    Returns both a summary recall_df and a debug_df with normalized text and embeddings.
    """

    recall_rows = []
    debug_rows = []

    for sample in tqdm(dataset):
        question = sample.get("user_input", "N/A")
        retrieved_raw = sample.get(top_k_context_key, [])
        gold_raw = sample.get(gold_context_key, [])

        # Normalize texts
        retrieved_norm = [normalize_text(r) for r in retrieved_raw if isinstance(r, str)]
        gold_norm = [normalize_text(g) for g in gold_raw if isinstance(g, str)]

        # Compute embeddings
        retrieved_emb = []
        gold_emb = []
        sim_matrix = []
        emb_hits = 0

        if method in {"embedding", "both"} and gold_norm and retrieved_norm:
            try:
                gold_emb = np.array(embedder.embed_documents(gold_norm)).astype("float32")
                retrieved_emb = np.array(embedder.embed_documents(retrieved_norm)).astype("float32")
                sim_matrix = cosine_similarity(gold_emb, retrieved_emb)
                emb_hits = sum(np.any(sim_row >= sim_threshold) for sim_row in sim_matrix)
            except Exception as e:
                print(f"Embedding error for question: {question}\n{e}")

        # Exact match hits
        em_hits = 0
        if method in {"exact", "both"}:
            em_hits = sum(
                any(fuzz.partial_ratio(g, r) > 85 for r in retrieved_norm)
                for g in gold_norm
            )

        total = len(gold_norm)
        recall_em = em_hits / total if total else 0
        recall_emb = emb_hits / total if total else 0

        # === Store evaluation results ===
        recall_rows.append({
            "question": question,
            "total_gold": total,
            "recall_em": recall_em,
            "recall_emb": recall_emb,
            "missing_em": [g for g in gold_norm if not any(fuzz.partial_ratio(g, r) > 85 for r in retrieved_norm)],
            "missing_emb": [
                g for idx, g in enumerate(gold_norm)
                if method in {"embedding", "both"} and (
                    idx >= len(sim_matrix) or not np.any(sim_matrix[idx] >= sim_threshold)
                )
            ]
        })

        # === Store debug info ===
        debug_rows.append({
            "question": question,
            "retrieved_norm": retrieved_norm,
            "retrieved_emb": retrieved_emb.tolist() if len(retrieved_emb) else [],
            "reference_norm": gold_norm,
            "reference_emb": gold_emb.tolist() if len(gold_emb) else []
        })

    recall_df = pd.DataFrame(recall_rows)
    debug_df = pd.DataFrame(debug_rows)
    return recall_df, debug_df

In [41]:
# load ragas eval dataset from json
with open("/Users/alex/Documents/Data Science Master/thesis_RAG/data/database_05_08/ragas_eval_dataset.json", "r") as f:
    ragas_eval_dataset = json.load(f)

recall_df, debug_df = context_recall_with_reference(
    dataset=ragas_eval_dataset,
    embedder=embedder,
    method="both",
    sim_threshold=0.75
)

100%|██████████| 149/149 [01:32<00:00,  1.62it/s]


In [43]:
recall_df.head(50)

,question,total_gold,recall_em,recall_emb,missing_em,missing_emb
0,"GM operating margin 2023 vs 2022, GM.",0,0.0,0.0,[],[]
1,"NGC's cyber investments boost investor confidence, enhance valuation, and bolster stability.",0,0.0,0.0,[],[]
2,"SBA Comm., credit evals & DTA quality receivables.",0,0.0,0.0,[],[]
3,"Q: For GPN, what was the fair value of share awards vested in 2009?\nA: 6.2\nQ: what was the value in 2007?\nA: 1.7\nQ: what was the net change in value?\nA: A0\nQ: what is the net change divided by the 2007 value?",1,1.0,1.0,[],[]
4,what was the cost per tower in American Tower’s colombia movil acquisition?,1,1.0,1.0,[],[]
5,Q: what is the implicit interest cost rate reported by Entergy in 2016?\nA: 5.13\nQ: what is the rate divided by 100?\nA: A0\nQ: what is the value of lease payments due after 2021?\nA: 257812\nQ: what is the product of the rate and those payments?,2,0.5,1.0,[the years thereafter of amount in thousands is 257812],[]
6,R&D expense as a % of net product revs for VRTX in 2023.,0,0.0,0.0,[],[]
7,% of part-time workforce and flexibility in labor costs for SPG.,0,0.0,0.0,[],[]
8,Competitor mix impact on pricing & positioning for NRG Energy (NRG).,0,0.0,0.0,[],[]
9,"For Citigroup, what was the percentage discount given in the reset of convertible preferred stock issued in the private offering ?",1,0.0,1.0,[on january 23 2009 pursuant to our prior agreement with the purchasers of the 12.5 billion convertible preferred stock issued in the private offering the conversion price was reset from 31.62 per share to 26.35 per share .],[]


In [46]:
# Filter out rows where there are no gold contexts
valid_rows = recall_df[recall_df["total_gold"] > 0]

# Compute the mean recall for exact match and embedding match
mean_recall_em = valid_rows["recall_em"].mean()
mean_recall_emb = valid_rows["recall_emb"].mean()

print(f"Mean Exact Match Recall: {mean_recall_em:.4f}")
print(f"Mean Embedding Recall:   {mean_recall_emb:.4f}")

Mean Exact Match Recall: 0.4774
Mean Embedding Recall:   1.0000


In [44]:
recall_df.to_csv("recall_results.csv", index=False)
debug_df.to_json("recall_debug.json", orient="records", indent=2)

# Context Precision

In [100]:
def context_precision_with_precomputed_embeddings(
    dataset: List[Dict],
    sim_threshold: float = 0.85
) -> pd.DataFrame:
    """
    Compute context precision using precomputed embeddings for non-FinDER sources only.
    """
    results = []

    for sample in tqdm(dataset):
        if sample.get("source") == "FinDER":
            continue  # Skip FinDER samples

        question = sample.get("question", "N/A")
        retrieved = sample.get("retrieved_norm", [])
        gold = sample.get("reference_norm", [])
        retrieved_emb = sample.get("retrieved_emb", [])
        gold_emb = sample.get("reference_emb", [])

        precision_emb = 0
        irrelevant_emb = []

        try:
            if gold_emb and retrieved_emb:
                retrieved_matrix = np.array(retrieved_emb).astype("float32")
                gold_matrix = np.array(gold_emb).astype("float32")
                sim_matrix = cosine_similarity(retrieved_matrix, gold_matrix)
                hits = np.sum(np.any(sim_matrix >= sim_threshold, axis=1))
                precision_emb = hits / len(retrieved_matrix) if len(retrieved_matrix) else 0
                irrelevant_emb = [retrieved[i] for i in range(len(retrieved))
                                  if not np.any(sim_matrix[i] >= sim_threshold)]
        except Exception as e:
            print(f"Embedding mismatch or error on: {question}\n{e}")

        results.append({
            "id": sample.get("ID", "N/A"),
            "source": sample.get("source", "N/A"),
            "question": question,
            "total_retrieved": len(retrieved),
            "precision_emb": precision_emb,
            "irrelevant_emb": irrelevant_emb
        })

    return pd.DataFrame(results)

In [76]:
import pandas as pd
import json

# Load RAGAS debug dataset
with open("/Users/alex/Documents/Data Science Master/thesis_RAG/notebooks/recall_debug.json") as f:
    ragas_debug_data = json.load(f)

# Load metadata-enhanced original RAGAS eval file
with open("/Users/alex/Documents/Data Science Master/thesis_RAG/data/database_05_08/final_ragas_eval_with_metadata.json") as f:
    ragas_metadata_data = json.load(f)

In [77]:
ragas_debug_df = pd.DataFrame(ragas_debug_data)
ragas_metadata_df = pd.DataFrame(ragas_metadata_data)

# Ensure clean question matching
ragas_debug_df["clean_question"] = ragas_debug_df["question"].str.lower().str.strip()
ragas_metadata_df["clean_question"] = ragas_metadata_df["user_input"].str.lower().str.strip()

# Merge to get source
merged_df = ragas_debug_df.merge(
    ragas_metadata_df[["ID", "source", "clean_question"]],
    on="clean_question",
    how="left"
)

In [78]:
merged_df

question  \
0                                                                                                                                                                                     GM operating margin 2023 vs 2022, GM.   
1                                                                                                                              NGC's cyber investments boost investor confidence, enhance valuation, and bolster stability.   
2                                                                                                                                                                        SBA Comm., credit evals & DTA quality receivables.   
3    Q: For GPN, what was the fair value of share awards vested in 2009?\nA: 6.2\nQ: what was the value in 2007?\nA: 1.7\nQ: what was the net change in value?\nA: A0\nQ: what is the net change divided by the 2007 value?   
4                                                                                                                                               what was the cost per tower in American Tower’s colombia movil acquisition?   
..                                                                                                                                                                                                                      ...   
144                                                                                                                                                      For AMT, what was the average rental expense between 2002 and 2004   
145                                                                                                                                          For AON, what was the percent of the increase in the revenue from 2010 to 2011   
146                                                                                                                          For BDX, what was the percent of the total international facilities square feet of owned by bd   
147                                                                                                                           For CAT, what is the expected growth rate in pension and opb contributions from 2017 to 2018?   
148                                                                                              For EOG, considering the twelve months ended december 31 , 2017 , what was the percentual increase observed in total puds?   

                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                              

In [98]:
merged_df.columns

Index(['ID', 'question', 'retrieved_norm', 'retrieved_emb', 'reference_norm',
       'reference_emb', 'source'],
      dtype='object')

In [97]:
#merged_df.drop(columns=["clean_question"], inplace=True)

# Reorder columns with 'ID' first
columns = ['ID'] + [col for col in merged_df.columns if col != 'ID']
merged_df = merged_df[columns]

In [79]:
non_finder_samples = merged_df[merged_df["source"] != "FinDER"].to_dict(orient="records")

In [101]:
precision_df = context_precision_with_precomputed_embeddings(non_finder_samples)
print(f"Mean Precision (excluding FinDER): {precision_df['precision_emb'].mean():.4f}")

100%|██████████| 81/81 [00:00<00:00, 975.07it/s]

Mean Precision (excluding FinDER): 0.3000


In [103]:
# precision df to csv
precision_df.to_csv("finqa_precision.csv", index=False)

In [102]:
precision_df.head()

id     source  \
0   GPN/2009/page_85.pdf  ConvFinQA   
1  AMT/2012/page_123.pdf      FinQA   
2  ETR/2016/page_175.pdf  ConvFinQA   
3    C/2008/page_100.pdf      FinQA   
4   IPG/2014/page_37.pdf  ConvFinQA   

                                                                                                                                                                                                                                                  question  \
0                                   Q: For GPN, what was the fair value of share awards vested in 2009?\nA: 6.2\nQ: what was the value in 2007?\nA: 1.7\nQ: what was the net change in value?\nA: A0\nQ: what is the net change divided by the 2007 value?   
1                                                                                                                                                                              what was the cost per tower in American Tower’s colombia movil acquisition?   
2  Q: what is the implicit interest cost rate reported by Entergy in 2016?\nA: 5.13\nQ: what is the rate divided by 100?\nA: A0\nQ: what is the value of lease payments due after 2021?\nA: 257812\nQ: what is the product of the rate and those payments?   
3                                                                                                                       For Citigroup, what was the percentage discount given in the reset of convertible preferred stock issued in the private offering ?   
4                                                                                   Q: For IPG, what was the total amount of long-term debt in the years of 2013 and 2014, combined?\nA: A0\nQ: and what was the total amount of debt in that same period?   

   total_retrieved  precision_emb  \
0               10            0.4   
1               10            0.1   
2               10            0.3   
3               10            0.4   
4               10            1.0   

                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                       

# F1 Score

In [104]:
def compute_f1_from_precision_recall_dfs(
    precision_df: pd.DataFrame,
    recall_df: pd.DataFrame,
    precision_col: str = "precision_emb",
    recall_col: str = "recall_emb"
) -> pd.DataFrame:
    """
    Combines precision and recall DataFrames to compute F1 score per sample.

    Args:
        precision_df: Output of context_precision_with_reference()
        recall_df: Output of context_recall_with_reference()
        precision_col: Column name for precision score
        recall_col: Column name for recall score

    Returns:
        A merged DataFrame with F1 score added
    """
    # Join on 'question'
    merged = pd.merge(precision_df, recall_df, on="question", suffixes=("_prec", "_rec"))

    def safe_f1(p, r):
        return (2 * p * r / (p + r)) if (p + r) > 0 else 0.0

    merged["f1"] = merged.apply(lambda row: safe_f1(row[precision_col], row[recall_col]), axis=1)

    return merged

In [112]:
import pandas as pd

# Step 1: Load your precision and recall results from CSVs
precision_df = pd.read_csv("/Users/alex/Documents/Data Science Master/thesis_RAG/notebooks/Eval Results/precision.csv")  # output of context_precision_with_reference
recall_df = pd.read_csv("/Users/alex/Documents/Data Science Master/thesis_RAG/notebooks/Eval Results/recall_results.csv")        # output of context_recall_with_reference

# Step 2: Compute F1 scores

f1_df = compute_f1_from_precision_recall_dfs(
    precision_df=precision_df,
    recall_df=recall_df,
    precision_col="precision_emb",  # change if your column has a different name
    recall_col="recall_emb"
)

# Step 3: View or save the result
print(f1_df[["question", "precision_emb", "recall_emb", "f1"]].head())
f1_df.to_csv("f1_results.csv", index=False)

                                                                                                                                                                                                                                                  question  \
0                                   Q: For GPN, what was the fair value of share awards vested in 2009?\nA: 6.2\nQ: what was the value in 2007?\nA: 1.7\nQ: what was the net change in value?\nA: A0\nQ: what is the net change divided by the 2007 value?   
1                                                                                                                                                                              what was the cost per tower in American Tower’s colombia movil acquisition?   
2  Q: what is the implicit interest cost rate reported by Entergy in 2016?\nA: 5.13\nQ: what is the rate divided by 100?\nA: A0\nQ: what is the value of lease payments due after 2021?\nA: 257812\nQ: what is the product of the rate and tho

In [111]:
mean_f1 = f1_df["f1"].mean()
print("Mean F1 Score:", round(mean_f1, 4))

Mean F1 Score: 0.3829


# Context Entity Recall

In [13]:
from typing import List, Dict
import pandas as pd
from tqdm import tqdm

def context_entity_recall(
    dataset: List[Dict],
    nlp,
    top_k_context_key: str = "retrieved_contexts",
    gold_context_key: str = "reference_contexts"
) -> pd.DataFrame:
    """
    Computes entity-level recall: proportion of gold named entities found in retrieved contexts.
 
    Args:
        dataset: List of samples with user_input, retrieved_contexts, and reference_contexts
        nlp: spaCy language model for NER
        top_k_context_key: Key for retrieved contexts
        gold_context_key: Key for reference (gold) contexts
 
    Returns:
        pd.DataFrame with entity recall metrics and diagnostics
    """
    results = []
 
    for sample in tqdm(dataset):
        question = sample.get("user_input", "N/A")
        retrieved = sample.get(top_k_context_key, [])
        gold = sample.get(gold_context_key, [])
 
        # Join all contexts into one string each
        retrieved_text = " ".join([r for r in retrieved if isinstance(r, str)])
        gold_text = " ".join([g for g in gold if isinstance(g, str)])
 
        # Run NER
        retrieved_doc = nlp(retrieved_text)
        gold_doc = nlp(gold_text)
 
        # Extract unique entities (as strings)
        retrieved_ents = set(ent.text.strip().lower() for ent in retrieved_doc.ents)
        gold_ents = set(ent.text.strip().lower() for ent in gold_doc.ents)
 
        # Calculate overlap
        common_ents = retrieved_ents & gold_ents
        total_gold_ents = len(gold_ents)
        recall = len(common_ents) / total_gold_ents if total_gold_ents else 0
 
        results.append({
            "ID": sample.get("ID", "N/A"),
            "source": sample.get("source", "N/A"),
            "reasoning_type": sample.get("reasoning_type", "N/A"),
            "question": question,
            "retrieved_entities": list(retrieved_ents),
            "gold_entities": list(gold_ents),
            "common_entities": list(common_ents),
            "num_common_entities": len(common_ents),
            "total_gold_entities": total_gold_ents,
            "entity_recall": recall
        })
 
    return pd.DataFrame(results)

In [14]:
import spacy
import json

# Load spaCy model
nlp = spacy.load("en_core_web_sm")

# Load dataset
with open("/Users/alex/Documents/Data Science Master/thesis_RAG/notebooks/Eval Results/final_ragas_eval_with_metadata.json") as f:
    dataset = json.load(f)

# Filter out FinDER samples
filtered_dataset = [sample for sample in dataset if sample.get("source") != "FinDER"]

# Run entity-level recall
entity_recall_df = context_entity_recall(filtered_dataset, nlp)

# Optional: view or save
print(entity_recall_df.head())
entity_recall_df.to_csv("entity_recall_results.csv", index=False)

100%|██████████| 81/81 [00:54<00:00,  1.48it/s]

                      ID     source reasoning_type  \
0   GPN/2009/page_85.pdf  ConvFinQA  Compositional   
1  AMT/2012/page_123.pdf      FinQA  Compositional   
2  ETR/2016/page_175.pdf  ConvFinQA  Compositional   
3    C/2008/page_100.pdf      FinQA  Compositional   
4   IPG/2014/page_37.pdf  ConvFinQA  Compositional   

                                            question  \
0  Q: For GPN, what was the fair value of share a...   
1  what was the cost per tower in American Tower’...   
2  Q: what is the implicit interest cost rate rep...   
3  For Citigroup, what was the percentage discoun...   
4  Q: For IPG, what was the total amount of long-...   

                                  retrieved_entities  \
0  [2.247w, the end of the three-year, $ 11 milli...   
1  [$ 174.4 million, 6, 201cmillicom, 302.6, 12.8...   
2  [6, 2.0, 23.2, 8.09%, michigan, one, 10.2, the...   
3  [1,200.0, up to $ 1 billion, 0.983%\t\t1.1%, c...   
4  [2,500, 2,087, june 2029\nsemi-annually, 6, ra...   

 

In [17]:
mean_entity_recall = entity_recall_df["entity_recall"].mean()
print(f"Mean Entity Recall: {mean_entity_recall:.4f}")

Mean Entity Recall: 0.6490


In [22]:
def compute_ndcg_with_embeddings(
    dataset: List[Dict],
    sim_threshold: float = 0.85,
    k: int = 10
) -> pd.DataFrame:
    """
    Compute nDCG@k using precomputed embeddings and skip FinDER samples.
    """
    results = []

    for sample in tqdm(dataset):
        if not sample.get("reference_emb"):
            continue

        question = sample.get("user_input", "N/A")
        retrieved = sample.get("retrieved_norm", [])[:k]
        gold = sample.get("reference_norm", [])
        retrieved_emb = sample.get("retrieved_emb", [])
        gold_emb = sample.get("reference_emb", [])

        relevance = [0] * len(retrieved)

        if gold_emb and retrieved_emb:
            try:
                retrieved_matrix = np.array(retrieved_emb[:k]).astype("float32")
                gold_matrix = np.array(gold_emb).astype("float32")
                sim_matrix = cosine_similarity(retrieved_matrix, gold_matrix)
                for i, sim_row in enumerate(sim_matrix):
                    if np.any(sim_row >= sim_threshold):
                        relevance[i] = 1
            except Exception as e:
                print(f"Embedding error for question: {question}\n{e}")

        # Compute DCG and IDCG
        dcg = sum(rel / np.log2(i + 2) for i, rel in enumerate(relevance))
        ideal_relevance = sorted(relevance, reverse=True)
        idcg = sum(rel / np.log2(i + 2) for i, rel in enumerate(ideal_relevance))
        ndcg = dcg / idcg if idcg > 0 else 0.0

        results.append({
            "question": question,
            "dcg": dcg,
            "idcg": idcg,
            f"ndcg@{k}": ndcg
        })

    df = pd.DataFrame(results)
    df[f"mean_ndcg@{k}"] = df[f"ndcg@{k}"].mean()
    return df

In [25]:
import json
import numpy as np
from sklearn.metrics.pairwise import cosine_similarity

with open("/Users/alex/Documents/Data Science Master/thesis_RAG/notebooks/Eval Results/retrieved_embeddings.json") as f:
    dataset = json.load(f)

ndcg_df = compute_ndcg_with_embeddings(dataset, sim_threshold=0.75, k=10)
print(f"Mean nDCG@10: {ndcg_df['ndcg@10'].mean():.4f}")

100%|██████████| 149/149 [00:00<00:00, 2853.58it/s]

Mean nDCG@10: 0.9974


In [26]:
for th in [0.6, 0.7, 0.75, 0.8, 0.85, 0.9]:
    df = compute_ndcg_with_embeddings(dataset, sim_threshold=th)
    print(f"Mean nDCG@10 (threshold={th}): {df[f'ndcg@10'].mean():.4f}")

100%|██████████| 149/149 [00:00<00:00, 2758.50it/s]


Mean nDCG@10 (threshold=0.6): 1.0000


100%|██████████| 149/149 [00:00<00:00, 2172.19it/s]


Mean nDCG@10 (threshold=0.7): 1.0000


100%|██████████| 149/149 [00:00<00:00, 2998.89it/s]


...

Mean nDCG@10 (threshold=0.8): 0.9101


100%|██████████| 149/149 [00:00<00:00, 3037.52it/s]


Mean nDCG@10 (threshold=0.85): 0.6196


100%|██████████| 149/149 [00:00<00:00, 2946.63it/s]

Mean nDCG@10 (threshold=0.9): 0.1448


In [ ]:
import spacy
import json

# Load spaCy model (use a financial NER model if available; otherwise default to en_core_web_lg)
nlp = spacy.load("en_core_web_lg")

# Load your RAGAS dataset (same format as previous evaluations)
with open("/path/to/ragas_eval_dataset_with_embeddings.json") as f:
    ragas_data = json.load(f)

# Run entity recall evaluation
entity_df = context_entity_recall(ragas_data, nlp)

# Save to file
entity_df.to_csv("entity_recall_results.csv", index=False)

# Optionally view basic stats
print("Mean Entity Recall:", entity_df["entity_recall"].mean())

In [ ]:
import numpy as np
import pandas as pd
from typing import List, Dict
from tqdm import tqdm
from sklearn.metrics.pairwise import cosine_similarity

def context_recall_with_precomputed_embeddings(
    dataset: List[Dict],
    sim_threshold: float = 0.85
) -> pd.DataFrame:
    """
    Compute context recall using precomputed embeddings.
    """
    results = []

    for sample in tqdm(dataset):
        question = sample.get("question", "N/A")
        source = sample.get("source", "N/A")
        sample_id = sample.get("ID", "N/A")
        retrieved = sample.get("retrieved_norm", [])
        gold = sample.get("reference_norm", [])
        retrieved_emb = sample.get("retrieved_emb", [])
        gold_emb = sample.get("reference_emb", [])

        recall_emb = 0
        missing_emb = []

        try:
            if gold_emb and retrieved_emb:
                gold_matrix = np.array(gold_emb).astype("float32")
                retrieved_matrix = np.array(retrieved_emb).astype("float32")
                sim_matrix = cosine_similarity(gold_matrix, retrieved_matrix)
                hits = np.sum(np.any(sim_matrix >= sim_threshold, axis=1))
                recall_emb = hits / len(gold_matrix) if len(gold_matrix) else 0
                missing_emb = [gold[i] for i in range(len(gold))
                               if not np.any(sim_matrix[i] >= sim_threshold)]
        except Exception as e:
            print(f"Embedding error for question: {question}\n{e}")

        results.append({
            "ID": sample_id,
            "source": source,
            "question": question,
            "total_gold": len(gold),
            "recall_emb": recall_emb,
            "missing_emb": missing_emb
        })

    return pd.DataFrame(results)


In [ ]:
from typing import List, Dict
import numpy as np
import pandas as pd
from tqdm import tqdm
from sklearn.metrics.pairwise import cosine_similarity

def context_recall_with_reference(
    dataset: List[Dict],
    embedder,
    top_k_context_key: str = "retrieved_contexts",
    gold_context_key: str = "reference_contexts",
    method: str = "embedding",  # "exact", "embedding", or "both"
    sim_threshold: float = 0.85
) -> pd.DataFrame:
    """
    Evaluates context recall using exact match or embedding similarity.
    Uses precomputed embeddings if available in the sample.
    """
    results = []

    for sample in tqdm(dataset):
        question = sample.get("user_input", "N/A")
        retrieved = sample.get(top_k_context_key, [])
        gold = sample.get(gold_context_key, [])

        retrieved = [r.lower().strip() for r in retrieved if isinstance(r, str)]
        gold = [g.lower().strip() for g in gold if isinstance(g, str)]

        em_hits = 0
        emb_hits = 0
        sim_matrix = []

        # === Exact Match ===
        if method in {"exact", "both"}:
            em_hits = sum(any(g in r for r in retrieved) for g in gold)

        # === Embedding Similarity ===
        if method in {"embedding", "both"} and gold and retrieved:
            try:
                # Use precomputed embeddings if available
                if "retrieved_emb" in sample and "reference_emb" in sample:
                    retrieved_emb = np.array(sample["retrieved_emb"]).astype("float32")
                    gold_emb = np.array(sample["reference_emb"]).astype("float32")
                else:
                    gold_emb = np.array(embedder.embed_documents(gold)).astype("float32")
                    retrieved_emb = np.array(embedder.embed_documents(retrieved)).astype("float32")

                sim_matrix = cosine_similarity(gold_emb, retrieved_emb)
                emb_hits = sum(np.any(sim_row >= sim_threshold) for sim_row in sim_matrix)

            except Exception as e:
                print(f"⚠️ Embedding error for question: {question}\n{e}")

        total = len(gold)
        recall_em = em_hits / total if total else 0
        recall_emb = emb_hits / total if total else 0

        results.append({
            "id": sample.get("id", "N/A"),
            "source": sample.get("source", "N/A"),
            "question": question,
            "total_gold": total,
            "recall_em": recall_em,
            "recall_emb": recall_emb,
            "missing_em": [g for g in gold if not any(g in r for r in retrieved)],
            "missing_emb": [
                g for idx, g in enumerate(gold)
                if method in {"embedding", "both"} and (
                    idx >= len(sim_matrix) or not np.any(sim_matrix[idx] >= sim_threshold)
                )
            ]
        })

    return pd.DataFrame(results)

In [18]:
def evaluate_reranker(gold_data, top_k_faiss=50, top_k_reranked=10):
    reranker = CrossEncoderReranker()  # your existing reranker class
    results = []

    for sample in tqdm(gold_data):
        question = sample["question"]
        gold_contexts = sample.get("gold_context", {})
        gold_contexts = list(gold_contexts.values()) if isinstance(gold_contexts, dict) else [gold_contexts]
        gold_contexts = [g.lower().strip() for g in gold_contexts if isinstance(g, str)]

        # FAISS
        q_embed = np.array(init_embedder().embed_query(question)).astype("float32").reshape(1, -1)
        faiss.normalize_L2(q_embed)
        D, I = index.search(q_embed, top_k_faiss)

        candidate_docs = [
            metadata_dict[chunk_ids[i]]
            for i in I[0]
            if chunk_ids[i] in metadata_dict and "text" in metadata_dict[chunk_ids[i]]
        ]
        reranked = reranker.rerank(question, candidate_docs, top_k=top_k_reranked)
        reranked_texts = [doc["text"].lower() for doc in reranked]

        found = any(g in r for g in gold_contexts for r in reranked_texts)

        results.append({
            "question": question,
            "gold_found_after_rerank": found,
            "gold_context": gold_contexts[0][:120] if gold_contexts else None,
            "gold_context_rank_post_rerank": next((i for i, r in enumerate(reranked_texts) if any(g in r for g in gold_contexts)), -1)
        })

    return pd.DataFrame(results)

In [19]:
df_reranked = evaluate_reranker(gold_data)
df_reranked.to_csv("reranker_eval_r10.csv", index=False)

hits = (df_reranked["gold_found_after_rerank"] == True).sum()
print(f"\nReranker Recall@10: {hits}/{len(df_reranked)} → {100 * hits / len(df_reranked):.2f}%")

  0%|          | 0/149 [00:00<?, ?it/s]


NameError: name 'chunked' is not defined

In [46]:
from typing import List, Dict
import pandas as pd
from tqdm import tqdm

def evaluate_context_recall(
    dataset: List[Dict],
    top_k_context_key: str = "retrieved_contexts",
    gold_context_key: str = "reference_contexts"
) -> pd.DataFrame:
    """
    Evaluates context recall for each QA sample in a dataset.
    
    Args:
        dataset: List of dicts. Each dict should include:
            - "user_input": question
            - top_k_context_key: list of retrieved context strings
            - gold_context_key: list of gold/reference context strings
        top_k_context_key: name of field containing retrieved contexts
        gold_context_key: name of field containing gold/reference contexts
        
    Returns:
        DataFrame with question, context recall, and detailed hits
    """
    results = []

    for sample in tqdm(dataset):
        question = sample.get("user_input", "N/A")
        retrieved = sample.get(top_k_context_key, [])
        gold = sample.get(gold_context_key, [])

        # Normalize
        retrieved = [r.lower().strip() for r in retrieved]
        gold = [g.lower().strip() for g in gold]

        hits = sum(any(g in r for r in retrieved) for g in gold)
        total = len(gold)
        recall = hits / total if total > 0 else 0.0

        results.append({
            "question": question,
            "context_recall": recall,
            "total_gold": total,
            "hits_found": hits,
            "missing_gold": [g for g in gold if not any(g in r for r in retrieved)]
        })

    return pd.DataFrame(results)

In [51]:
context_recall_df = evaluate_context_recall(finqa_eval_dataset)

100%|██████████| 2/2 [00:00<00:00, 295.99it/s]


In [52]:
print(context_recall_df["context_recall"].mean())

1.0


In [ ]:
from typing import List, Dict
import pandas as pd
import numpy as np
from tqdm import tqdm
from sklearn.metrics.pairwise import cosine_similarity

def evaluate_context_recall(
    dataset: List[Dict],
    embedder,
    top_k_context_key: str = "retrieved_contexts",
    gold_context_key: str = "reference_contexts",
    method: str = "embedding",  # "exact", "embedding", or "both"
    sim_threshold: float = 0.85
) -> pd.DataFrame:
    """
    Evaluates context recall via exact match or embedding-based similarity.

    Args:
        dataset: List of QA samples with user_input, retrieved_contexts, reference_contexts
        embedder: an object with .embed_documents() method (e.g., OpenAIEmbedder or SentenceTransformer)
        method: "exact", "embedding", or "both"
        sim_threshold: similarity threshold for embedding-based match

    Returns:
        pd.DataFrame with recall scores and detailed diagnostics
    """
    results = []

    for sample in tqdm(dataset):
        question = sample.get("user_input", "N/A")
        retrieved = sample.get(top_k_context_key, [])
        gold = sample.get(gold_context_key, [])

        retrieved = [r.lower().strip() for r in retrieved if isinstance(r, str)]
        gold = [g.lower().strip() for g in gold if isinstance(g, str)]

        em_hits = 0
        emb_hits = 0

        # === Exact Match ===
        if method in {"exact", "both"}:
            em_hits = sum(any(g in r for r in retrieved) for g in gold)

        # === Embedding Similarity ===
        if method in {"embedding", "both"} and gold and retrieved:
            try:
                gold_emb = np.array(embedder.embed_documents(gold)).astype("float32")
                retrieved_emb = np.array(embedder.embed_documents(retrieved)).astype("float32")
                sim_matrix = cosine_similarity(gold_emb, retrieved_emb)
                emb_hits = sum(np.any(sim_row >= sim_threshold) for sim_row in sim_matrix)
            except Exception as e:
                print(f"Embedding error for question: {question}\n{e}")

        # Pick which metric to use for recall
        total = len(gold)
        recall_em = em_hits / total if total else 0
        recall_emb = emb_hits / total if total else 0

        results.append({
            "question": question,
            "total_gold": total,
            "recall_em": recall_em,
            "recall_emb": recall_emb,
            "missing_em": [g for g in gold if not any(g in r for r in retrieved)],
            "missing_emb": [g for idx, g in enumerate(gold)
                            if method in {"embedding", "both"} and
                            (idx >= len(sim_matrix) or not np.any(sim_matrix[idx] >= sim_threshold))]
        })

    return pd.DataFrame(results)

In [54]:
recall_df = evaluate_context_recall(
    dataset=finqa_eval_dataset,
    embedder=embedder,
    method="both",
    sim_threshold=0.85
)
print("Avg EM Recall:", recall_df["recall_em"].mean())
print("Avg Embedding Recall:", recall_df["recall_emb"].mean())

100%|██████████| 2/2 [00:02<00:00,  1.30s/it]

Avg EM Recall: 1.0
Avg Embedding Recall: 1.0


In [56]:
def context_precision_with_reference(
    dataset: List[Dict],
    embedder,
    top_k_context_key: str = "retrieved_contexts",
    gold_context_key: str = "reference_contexts",
    method: str = "embedding",  # "exact", "embedding", or "both"
    sim_threshold: float = 0.85
) -> pd.DataFrame:
    """
    Evaluates context precision via exact match or embedding-based similarity.
 
    Args:
        dataset: List of QA samples with user_input, retrieved_contexts, reference_contexts
        embedder: an object with .embed_documents() method
        method: "exact", "embedding", or "both"
        sim_threshold: similarity threshold for embedding-based match
 
    Returns:
        pd.DataFrame with precision scores and detailed diagnostics
    """
    results = []
 
    for sample in tqdm(dataset):
        question = sample.get("user_input", "N/A")
        retrieved = sample.get(top_k_context_key, [])
        gold = sample.get(gold_context_key, [])
 
        retrieved = [r.lower().strip() for r in retrieved if isinstance(r, str)]
        gold = [g.lower().strip() for g in gold if isinstance(g, str)]
 
        em_hits = 0
        emb_hits = 0
 
        # === Exact Match ===
        if method in {"exact", "both"}:
            em_hits = sum(any(g in r for g in gold) for r in retrieved)
 
        # === Embedding Similarity ===
        if method in {"embedding", "both"} and gold and retrieved:
            try:
                gold_emb = np.array(embedder.embed_documents(gold)).astype("float32")
                retrieved_emb = np.array(embedder.embed_documents(retrieved)).astype("float32")
                sim_matrix = cosine_similarity(retrieved_emb, gold_emb)
                emb_hits = sum(np.any(sim_row >= sim_threshold) for sim_row in sim_matrix)
            except Exception as e:
                print(f"Embedding error for question: {question}\n{e}")
 
        # Pick which metric to use for precision
        total = len(retrieved)
        precision_em = em_hits / total if total else 0
        precision_emb = emb_hits / total if total else 0
 
        results.append({
            "question": question,
            "total_retrieved": total,
            "precision_em": precision_em,
            "precision_emb": precision_emb,
            "irrelevant_em": [r for r in retrieved if not any(g in r for g in gold)],
            "irrelevant_emb": [r for idx, r in enumerate(retrieved)
                               if method in {"embedding", "both"} and
                               (idx >= len(sim_matrix) or not np.any(sim_matrix[idx] >= sim_threshold))]
        })
 
    return pd.DataFrame(results)

In [57]:
# Evaluate context precision
precision_results = context_precision_with_reference(
    dataset=finqa_eval_dataset,
    embedder=embedder,
    method="both",  # or "embedding" or "exact"
    sim_threshold=0.85
)

# Summary
print("\n📊 Context Precision Summary:")
print(f"Average Exact Match Precision:     {precision_results['precision_em'].mean():.4f}")
print(f"Average Embedding-Based Precision: {precision_results['precision_emb'].mean():.4f}")

# Optional: Save detailed results
precision_results.to_csv("finqa_context_precision_diagnostics.csv", index=False)

100%|██████████| 2/2 [00:03<00:00,  1.54s/it]


📊 Context Precision Summary:
Average Exact Match Precision:     0.0500
Average Embedding-Based Precision: 0.1500


In [ ]:
import asyncio
from typing import List, Dict, Any, Optional
import os
from dotenv import load_dotenv
from ragas import EvaluationDataset, evaluate
import re

def normalize(text: str) -> str:
    """
    Normalize text for evaluation by:
    - lowercasing
    - removing excess whitespace
    - optional: removing punctuation or %/$
    """
    if not isinstance(text, str):
        return ""
    text = text.lower()
    text = re.sub(r'\s+', ' ', text)
    text = text.strip()
    return text

async def evaluate_ragas_dataset(
    dataset: List[Dict[str, Any]],
    metrics_list: Optional[List[str]] = None,
    llm_model: str = "gpt-4o-mini",
    llm_type: str = "openai",  # 'openai' or 'vllm'
    vllm_base_url: str = "http://localhost:8000/v1"
):
    """
    Evaluate a dataset using RAGAS with normalized inputs.

    Args:
        dataset: List of dicts with keys: user_input, retrieved_contexts, response, reference
        metrics_list: Which metrics to use
        llm_model: LLM name (e.g., 'gpt-4o')
        llm_type: 'openai' or 'vllm'
        vllm_base_url: if using vllm locally
    """
    from ragas.metrics import (
        LLMContextPrecisionWithReference,
        NonLLMContextPrecisionWithReference,
        LLMContextRecall,
        NonLLMContextRecall,
        ContextEntityRecall,
        Faithfulness,
        AnswerAccuracy,
        StringPresence,
    )
    available_metrics = {
        "context_precision_llm": LLMContextPrecisionWithReference,
        "context_precision_nonllm": NonLLMContextPrecisionWithReference,
        "context_recall_llm": LLMContextRecall,
        "context_recall_nonllm": NonLLMContextRecall,
        "context_entity_recall": ContextEntityRecall,
        "faithfulness": Faithfulness,
        "answer_accuracy": AnswerAccuracy,
        "string_presence": StringPresence,
    }

    if metrics_list is None:
        metrics_list = list(available_metrics.keys())

    metrics = [available_metrics[name]() for name in metrics_list if name in available_metrics]

    # === Normalize dataset ===
    normalized_dataset = []
    for row in dataset:
        norm_row = {
            "user_input": normalize(row.get("user_input", "")),
            "retrieved_contexts": [normalize(c) for c in row.get("retrieved_contexts", [])],
            "response": normalize(row.get("response", "")),
            "reference": normalize(row.get("reference", "")),
        }

        # Optional: normalize reference_contexts too
        if "reference_contexts" in row:
            norm_row["reference_contexts"] = [normalize(c) for c in row["reference_contexts"]]
        
        normalized_dataset.append(norm_row)

    # === Load LLM ===
    load_dotenv()
    if llm_type == "openai":
        from langchain_openai import ChatOpenAI
        from ragas.llms import LangchainLLMWrapper
        OPENAI_API_KEY = os.getenv("OPENAI_API_KEY")
        if not OPENAI_API_KEY:
            raise ValueError("OPENAI_API_KEY not found in .env file.")
        llm = ChatOpenAI(model=llm_model, api_key=OPENAI_API_KEY)
        evaluator_llm = LangchainLLMWrapper(llm)
    elif llm_type == "vllm":
        from langchain_community.llms import VLLMOpenAI
        from ragas.llms import LangchainLLMWrapper
        llm = VLLMOpenAI(model=llm_model, base_url=vllm_base_url)
        evaluator_llm = LangchainLLMWrapper(llm)
    else:
        raise ValueError(f"Unknown llm_type: {llm_type}")

    evaluation_dataset = EvaluationDataset.from_list(normalized_dataset)
    return evaluate(
        dataset=evaluation_dataset,
        metrics=metrics,
        llm=evaluator_llm
    )

In [58]:
import spacy
from typing import List, Dict
import pandas as pd
from tqdm import tqdm
 
def context_entity_recall(
    dataset: List[Dict],
    nlp,
    top_k_context_key: str = "retrieved_contexts",
    gold_context_key: str = "reference_contexts"
) -> pd.DataFrame:
    """
    Computes entity-level recall: proportion of gold named entities found in retrieved contexts.
 
    Args:
        dataset: List of samples with user_input, retrieved_contexts, and reference_contexts
        nlp: spaCy language model for NER
        top_k_context_key: Key for retrieved contexts
        gold_context_key: Key for reference (gold) contexts
 
    Returns:
        pd.DataFrame with entity recall metrics and diagnostics
    """
    results = []
 
    for sample in tqdm(dataset):
        question = sample.get("user_input", "N/A")
        retrieved = sample.get(top_k_context_key, [])
        gold = sample.get(gold_context_key, [])
 
        # Join all contexts into one string each
        retrieved_text = " ".join([r for r in retrieved if isinstance(r, str)])
        gold_text = " ".join([g for g in gold if isinstance(g, str)])
 
        # Run NER
        retrieved_doc = nlp(retrieved_text)
        gold_doc = nlp(gold_text)
 
        # Extract unique entities (as strings)
        retrieved_ents = set(ent.text.strip().lower() for ent in retrieved_doc.ents)
        gold_ents = set(ent.text.strip().lower() for ent in gold_doc.ents)
 
        # Calculate overlap
        common_ents = retrieved_ents & gold_ents
        total_gold_ents = len(gold_ents)
        recall = len(common_ents) / total_gold_ents if total_gold_ents else 0
 
        results.append({
            "question": question,
            "retrieved_entities": list(retrieved_ents),
            "gold_entities": list(gold_ents),
            "common_entities": list(common_ents),
            "num_common_entities": len(common_ents),
            "total_gold_entities": total_gold_ents,
            "entity_recall": recall
        })
 
    return pd.DataFrame(results)

In [59]:
import spacy

# Load spaCy English model (small is fast; medium or large is more accurate)
nlp = spacy.load("en_core_web_sm")

# Run entity-level recall evaluation
entity_results = context_entity_recall(
    dataset=finqa_eval_dataset,
    nlp=nlp,
    top_k_context_key="retrieved_contexts",
    gold_context_key="reference_contexts"
)

# Print summary
print(f"\n📊 Average Entity Recall: {entity_results['entity_recall'].mean():.4f}")

# Optional: Save diagnostics to CSV
entity_results.to_csv("finqa_entity_recall_diagnostics.csv", index=False)

100%|██████████| 2/2 [00:03<00:00,  1.52s/it]


📊 Average Entity Recall: 1.0000


In [60]:
def compute_f1_from_precision_recall_dfs(
    precision_df: pd.DataFrame,
    recall_df: pd.DataFrame,
    precision_col: str = "precision_emb",
    recall_col: str = "recall_emb"
) -> pd.DataFrame:
    """
    Combines precision and recall DataFrames to compute F1 score per sample.
 
    Args:
        precision_df: Output of context_precision_with_reference()
        recall_df: Output of context_recall_with_reference()
        precision_col: Column name for precision score
        recall_col: Column name for recall score
 
    Returns:
        A merged DataFrame with F1 score added
    """
    # Join on 'question'
    merged = pd.merge(precision_df, recall_df, on="question", suffixes=("_prec", "_rec"))
 
    def safe_f1(p, r):
        return (2 * p * r / (p + r)) if (p + r) > 0 else 0.0
 
    merged["f1"] = merged.apply(lambda row: safe_f1(row[precision_col], row[recall_col]), axis=1)
 
    return merged
 

In [62]:
# Step 1: Compute context precision and recall separately
precision_df = context_precision_with_reference(
    dataset=finqa_eval_dataset,
    embedder=embedder,
    method="embedding",   # or "both"
    sim_threshold=0.85
)

recall_df = evaluate_context_recall(
    dataset=finqa_eval_dataset,
    embedder=embedder,
    method="embedding",   # or "both"
    sim_threshold=0.85
)

# Step 2: Compute F1 score from the two
f1_df = compute_f1_from_precision_recall_dfs(
    precision_df=precision_df,
    recall_df=recall_df,
    precision_col="precision_emb",  # use "precision_em" if using exact match
    recall_col="recall_emb"         # or "recall_em"
)

# Step 3: Print summary
print(f"\n🔍 Average F1 Score: {f1_df['f1'].mean():.4f}")

# Step 4 (optional): Save to CSV
f1_df.to_csv("finqa_precision_recall_f1.csv", index=False)

100%|██████████| 2/2 [00:04<00:00,  2.04s/it]


🔍 Average F1 Score: 0.2476


In [63]:
from typing import List, Dict
import numpy as np
import pandas as pd
from sklearn.metrics.pairwise import cosine_similarity
from tqdm import tqdm
 
def mean_reciprocal_rank(
    dataset: List[Dict],
    embedder = None,
    top_k_context_key: str = "retrieved_contexts",
    gold_context_key: str = "reference_contexts",
    method: str = "embedding",  # "exact" or "embedding"
    sim_threshold: float = 0.85
) -> pd.DataFrame:
    """
    Calculates MRR for retrieved contexts given gold reference contexts.
 
    Returns:
        pd.DataFrame with reciprocal ranks and full MRR score
    """
    results = []
 
    for sample in tqdm(dataset):
        question = sample.get("user_input", "N/A")
        retrieved = sample.get(top_k_context_key, [])
        gold = sample.get(gold_context_key, [])
 
        retrieved = [r.lower().strip() for r in retrieved if isinstance(r, str)]
        gold = [g.lower().strip() for g in gold if isinstance(g, str)]
 
        rr = 0.0  # Reciprocal rank
 
        if method == "exact":
            for rank, ret in enumerate(retrieved, start=1):
                if any(g in ret for g in gold):
                    rr = 1.0 / rank
                    break
 
        elif method == "embedding" and embedder and gold and retrieved:
            try:
                gold_emb = np.array(embedder.embed_documents(gold)).astype("float32")
                retrieved_emb = np.array(embedder.embed_documents(retrieved)).astype("float32")
                sim_matrix = cosine_similarity(retrieved_emb, gold_emb)
                for rank, sim_row in enumerate(sim_matrix, start=1):
                    if np.any(sim_row >= sim_threshold):
                        rr = 1.0 / rank
                        break
            except Exception as e:
                print(f"Embedding error for question: {question}\n{e}")
 
        results.append({
            "question": question,
            "reciprocal_rank": rr
        })
 
    df = pd.DataFrame(results)
    df["mrr"] = df["reciprocal_rank"].mean()
    return df

In [64]:
df = mean_reciprocal_rank(finqa_eval_dataset, embedder=embedder)

print("MRR:", df['mrr'].iloc[0])

df.head()

 

100%|██████████| 2/2 [00:01<00:00,  1.00it/s]

MRR: 0.75


,question,reciprocal_rank,mrr
0,"Q: For GPN, what was the fair value of share a...",1.0,0.75
1,what was the cost per tower in American Tower’...,0.5,0.75


In [65]:
import numpy as np
import pandas as pd
from typing import List, Dict
from sklearn.metrics.pairwise import cosine_similarity
from tqdm import tqdm
 
def compute_ndcg(
    dataset: List[Dict],
    embedder = None,
    top_k_context_key: str = "retrieved_contexts",
    gold_context_key: str = "reference_contexts",
    method: str = "embedding",  # "exact" or "embedding"
    sim_threshold: float = 0.85,
    k: int = 10
) -> pd.DataFrame:
    """
    Compute nDCG@k for retrieved contexts.
 
    Returns:
        DataFrame with nDCG@k per sample and average
    """
    results = []
 
    for sample in tqdm(dataset):
        question = sample.get("user_input", "N/A")
        retrieved = sample.get(top_k_context_key, [])[:k]
        gold = sample.get(gold_context_key, [])
 
        retrieved = [r.lower().strip() for r in retrieved if isinstance(r, str)]
        gold = [g.lower().strip() for g in gold if isinstance(g, str)]
 
        relevance = [0] * len(retrieved)  # Relevance at each rank
 
        if method == "exact":
            for i, r in enumerate(retrieved):
                if any(g in r for g in gold):
                    relevance[i] = 1
 
        elif method == "embedding" and embedder and gold and retrieved:
            try:
                gold_emb = np.array(embedder.embed_documents(gold)).astype("float32")
                retrieved_emb = np.array(embedder.embed_documents(retrieved)).astype("float32")
                sim_matrix = cosine_similarity(retrieved_emb, gold_emb)
                for i, sim_row in enumerate(sim_matrix):
                    if np.any(sim_row >= sim_threshold):
                        relevance[i] = 1
            except Exception as e:
                print(f"Embedding error for question: {question}\n{e}")
 
        # === Compute DCG and nDCG ===
        dcg = sum(rel / np.log2(i + 2) for i, rel in enumerate(relevance))  # +2 because log2(i+1) with 0-indexing
        ideal_relevance = sorted(relevance, reverse=True)
        idcg = sum(rel / np.log2(i + 2) for i, rel in enumerate(ideal_relevance))
 
        ndcg = dcg / idcg if idcg > 0 else 0.0
 
        results.append({
            "question": question,
            "dcg": dcg,
            "idcg": idcg,
            "ndcg@{}".format(k): ndcg
        })
 
    df = pd.DataFrame(results)
    df["mean_ndcg"] = df["ndcg@{}".format(k)].mean()
    return df

In [66]:
ndcg_df = compute_ndcg(finqa_eval_dataset, embedder=embedder, k=10)
print("Average nDCG@10:", ndcg_df["mean_ndcg"].iloc[0])

100%|██████████| 2/2 [00:09<00:00,  4.79s/it]

Average nDCG@10: 0.7817254227631407
